In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import os

import pymesh
from headctools.preprocessing import utils
from headctools.tools.offline_augmentation import simulate_and_save

from src.deformetrica import estimate_registration

owrt = False  # Overwrite flag

# To check by the user
raw_images_path = os.path.expanduser(
    '~/Code/datasets/cq500/converted/selected/test'
)
fixed_image_path = 'CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz'

# To be set up automatically
preprocessed_images_path = os.path.join(raw_images_path,
                                        'preprocessed_ct_to_skull')
simulated_cran_path = os.path.join(preprocessed_images_path, 'cran')  # default
cran_imgs_only = None  # List of files with simulated craniectomy
registered_meshes_folder = os.path.join(
    preprocessed_images_path, f'reg_{os.path.split(fixed_image_path)}'
)
difference_meshes = os.path.join(preprocessed_images_path, 'diff')

## Image with defect registration test

Test-time mesh registration simulation:

1.  Grab an image without defect.
2.  Mesh and register the template to the skull.
3.  Simulate a craniectomy.
4.  Mesh the skull with the defect.
5.  Register (2) to (4).

### Simulate defect
Generate 1 simulation per img

In [2]:
simulate_and_save(preprocessed_images_path, out_folder=simulated_cran_path,
                  overwrite=owrt)
cran_imgs_only = [
    os.path.join(simulated_cran_path, f)
    for f in os.listdir(simulated_cran_path) if f.endswith('0_sim.nii.gz')
]

Folder: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull
Input: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-312_CT PRE CONTRAST THIN.nii.gz.
  File 0 already exists. Skipping.
Input: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-405_CT Thin Plain.nii.gz.
  File 0 already exists. Skipping.


### Convert to stl

In [3]:
utils.nii_to_stl_marching_cubes(preprocessed_images_path,
                                preprocessed_images_path, owrt)
cran_stl_files = utils.nii_to_stl_marching_cubes(cran_imgs_only,
                                                 simulated_cran_path, owrt)

    File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-312_CT PRE CONTRAST THIN.stl already exists. Skipping...
    File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/CQ500-CT-405_CT Thin Plain.stl already exists. Skipping...
    File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/CQ500-CT-312_CT PRE CONTRAST THIN0_sim.stl already exists. Skipping...
    File /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/CQ500-CT-405_CT Thin Plain0_sim.stl already exists. Skipping...


### Decimate meshes
Reduce the amount of points in each mesh to a 1 percent.

In [4]:
decimate_factor = 0.01

decimated_meshes_full = utils.decimate_meshes(preprocessed_images_path,
                                              decimate_factor, owrt)
decimated_meshes_def = utils.decimate_meshes(simulated_cran_path,
                                             decimate_factor, owrt)

Decimating meshes...
  Input folder: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull
  Found 2 meshes
    [1/2] Input mesh: CQ500-CT-312_CT PRE CONTRAST THIN.stl
    Mesh already exists, skipping...
    [2/2] Input mesh: CQ500-CT-405_CT Thin Plain.stl
    Mesh already exists, skipping...
Decimating meshes...
  Input folder: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran
  Found 2 meshes
    [1/2] Input mesh: CQ500-CT-312_CT PRE CONTRAST THIN0_sim.stl
    Mesh already exists, skipping...
    [2/2] Input mesh: CQ500-CT-405_CT Thin Plain0_sim.stl
    Mesh already exists, skipping...


### Register atlas to simulated defect images
Here the template image (the subject 312 with the full skull) will be registered to every skull with a defect.
For each patient with a defect, a folder will be created with two outputs:
 1) DeterministicAtlas__EstimatedParameters__Template_skull.vtk
 2) DeterministicAtlas__Reconstruction__skull__IMAGE_NAME.vtk

(1) Aligns better with the postoperative image if it's the same subject image, otherwise (2) will be the best adaptation of the template to this image.

In [5]:
cran_paths = [
    os.path.join(simulated_cran_path, f)
    for f in os.listdir(simulated_cran_path) if f.endswith('.vtk')
]
template_path = decimated_meshes_full[
    0]  # Preprocessed + dmtd complete skull (312)

# Fixed images will be all the patients with craniectomy. The template will
# adapt to each of these meshes. A folder for each patient will be saved.
paths_reg_deformetrica = []
paths_vtk_deformetrica = []
for fixed_path in cran_paths:
    out_path = os.path.join(simulated_cran_path, 'reg_' +
                            os.path.splitext(os.path.split(fixed_path)[1])[0]
                            )
    paths_reg_deformetrica.append(out_path)
    estimate_registration(template_path, [fixed_path], out_path,
                          overwrite=owrt)
    rec_file = [
        f for f in os.listdir(out_path)
        if f.startswith('DeterministicAtlas__Reconstruction__skull__subject_')
    ][0]
    defrm_file = os.path.join(out_path, rec_file)
    paths_vtk_deformetrica.append(defrm_file)

Output path: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc
  Output folder already exists and contains files. Please delete the folder or set the overwrite flag.
Output path: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-405_CT Thin Plain0_sim_decimated_1perc
  Output folder already exists and contains files. Please delete the folder or set the overwrite flag.


In [6]:
paths_vtk_deformetrica

['/home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc.stl',
 '/home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-405_CT Thin Plain0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-405_CT Thin Plain0_sim_decimated_1perc.vtk']

### Convert to stl
Needed for pymesh

In [7]:
paths_stl_deformetrica = utils.vtk_to_stl(paths_vtk_deformetrica)

Creating /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc.stl
Creating /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-405_CT Thin Plain0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-405_CT Thin Plain0_sim_decimated_1perc.stl


2022-09-26 15:59:57.483 (  16.283s) [        896BD180]      vtkDataReader.cxx:567    ERR| vtkGenericDataObjectReader (0x55a5de4b3840): Unrecognized file type: solid Visualization Toolkit generated SLA File for file: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc.stl
2022-09-26 15:59:57.483 (  16.283s) [        896BD180]      vtkDataReader.cxx:567    ERR| vtkGenericDataObjectReader (0x55a5de4b3840): Unrecognized file type: solid Visualization Toolkit generated SLA File for file: /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/cran/reg_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc/DeterministicAtlas__Reconstruction__skull__subject_CQ500-CT-312_CT PRE CONTRAST THIN0_sim_decimated_1perc.stl
2022-09-26 15:59:57.483 (  16.283s) [        896BD180]  

### Difference
Difference of the transformed template mesh with the craniectomy mesh

In [21]:
# Join both lists
meshes = zip(paths_stl_deformetrica, cran_stl_files)

In [20]:
list(meshes)

[]

In [19]:
os.makedirs(difference_meshes, exist_ok=True)

for full, inc in meshes:
    out_path = os.path.join(
        difference_meshes,
        os.path.split(inc)[1].split('_')[0].split('-')[-1] + '.stl'
    )
    full_mesh = pymesh.load_mesh(full)
    incomplete_mesh = pymesh.load_mesh(inc)
    output_mesh = pymesh.boolean(full_mesh, incomplete_mesh,
                                 operation="symmetric_difference")
    pymesh.save_mesh(out_path, output_mesh)
    print(f'saved {out_path}')

Input mesh is not PWN!


saved /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/diff/312.stl


Input mesh is not PWN!


saved /home/franco/Code/datasets/cq500/converted/selected/test/preprocessed_ct_to_skull/diff/405.stl
